In [1]:
pip install pandas matplotlib bar_chart_race

In [2]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [52]:
df = pd.read_csv("data/0_preprocessed.csv")
df.head(5)

,timestamp,duration,song,artist,album,song_artist
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,2019-07-05 03:01:44,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,2019-07-05 03:05:23,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,2019-07-05 03:08:41,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,2019-07-05 03:12:01,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker


In [41]:
df.isna().sum()

,0
ts,0
ms_played,0
master_metadata_track_name,52
master_metadata_album_artist_name,52
master_metadata_album_album_name,52


In [42]:
df = df.dropna()

In [43]:
df.dtypes

,0
ts,object
ms_played,int64
master_metadata_track_name,object
master_metadata_album_artist_name,object
master_metadata_album_album_name,object


In [44]:
tmp = pd.to_datetime(df['ts']).dt.strftime('%Y-%m-%d %H-%M-%S')
df['ts'] = pd.to_datetime(tmp, format='%Y-%m-%d %H-%M-%S')
df.head(5)

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name
0,2019-07-05 03:00:26,146830,7 rings,Ariana Grande,"thank u, next"
1,2019-07-05 03:01:44,77120,All Falls Down (feat. Juliander),Alan Walker,All Falls Down (feat. Juliander)
2,2019-07-05 03:05:23,219946,I Don't Care (with Justin Bieber),Ed Sheeran,I Don't Care (with Justin Bieber)
3,2019-07-05 03:08:41,197866,Beautiful People (feat. Khalid),Ed Sheeran,Beautiful People (feat. Khalid)
4,2019-07-05 03:12:01,199111,All Falls Down (feat. Juliander),Alan Walker,All Falls Down (feat. Juliander)


In [45]:
df['ms_played'] = round(df['ms_played'] / 60000, 2)
df['ms_played'].head(1)

,ms_played
0,2.45


In [46]:
import re

def shorten_string(text):
  # Remove substrings inside parentheses or brackets
  text = re.sub(r"\(.*?\)|\[.*?\]", "", text)
  # Trim the string
  text = text.strip()
  return text

df['master_metadata_track_name'] = df['master_metadata_track_name'].apply(shorten_string)
df.head(5)

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next"
1,2019-07-05 03:01:44,1.29,All Falls Down,Alan Walker,All Falls Down (feat. Juliander)
2,2019-07-05 03:05:23,3.67,I Don't Care,Ed Sheeran,I Don't Care (with Justin Bieber)
3,2019-07-05 03:08:41,3.30,Beautiful People,Ed Sheeran,Beautiful People (feat. Khalid)
4,2019-07-05 03:12:01,3.32,All Falls Down,Alan Walker,All Falls Down (feat. Juliander)


In [47]:
df['song_artist'] = df['master_metadata_track_name'] + ' - ' + df['master_metadata_album_artist_name']
df.head(1)

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,song_artist
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande


In [48]:
df.columns = ['timestamp', 'duration','song','artist','album','song_artist']
df.head(1)

,timestamp,duration,song,artist,album,song_artist
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande


In [17]:
from datetime import datetime, timedelta

# Create a list to store the datetime objects
timestamps = []

start_date = datetime(2019, 7, 5)
end_date = datetime(2024, 9, 6)

num_weeks = (end_date - start_date).days // 7

for i in range(num_weeks + 1):
  timestamps.append(start_date + timedelta(weeks=i))

# Print the list of datetime objects
print(len(timestamps), timestamps[0], timestamps[-1])

271 2019-07-05 00:00:00 2024-09-06 00:00:00


In [18]:
top_tracks_list = df['song_artist'].unique().tolist()

In [19]:
duration_dict_song = {song: [0 for _ in range(len(timestamps))] for song in top_tracks_list}
duration_total_dict_song = {song: 0 for song in top_tracks_list}

In [20]:
print(df['timestamp'][0], timestamps[0], df['timestamp'][0] < timestamps[0])

2019-07-05 03:00:26 2019-07-05 00:00:00 False


In [21]:
time_idx = 1
for i, row in df.iterrows():
  while row['timestamp'] > timestamps[time_idx]:
    for s in top_tracks_list:
      duration_dict_song[s][time_idx] = duration_total_dict_song[s]
    time_idx += 1

  song_artist = row['song_artist']
  duration_total_dict_song[song_artist] += row['duration']
  duration_dict_song[song_artist][time_idx] = duration_total_dict_song[song_artist]

In [22]:
df_pivot_song = pd.DataFrame(duration_dict_song, index=timestamps)
print(len(df_pivot_song), len(df_pivot_song.columns))
df_pivot_song

271 1493


,7 rings - Ariana Grande,All Falls Down - Alan Walker,I Don't Care - Ed Sheeran,Beautiful People - Ed Sheeran,Darkside - Alan Walker,Fractures - ILLENIUM,Gjøre Ingenting - Freddy Kalas,Goodbyes - Post Malone,bad guy - Billie Eilish,Delicate - Taylor Swift,...,Remember Me - Onionn Remix Countdown 2021 - Sơn Tùng M-TP,Nắng Ấm Xa Dần - Sơn Tùng M-TP,Cơn Mưa Ngang Qua - Sơn Tùng M-TP,Em Của Ngày Hôm Qua - Sơn Tùng M-TP,Constant Moderato - Arcade Player,Constant Moderato - Lofi Version - TiHA Studio,"Moon Halo - Honkai Impact 3Rd ""Everlasting Flames"" Animated Short Theme - HOYO-MiX",The Lion Sleeps Tonight - The Tokens,Hear Me - Ln,Chiếc Khăn Piêu - Dương Hoàng Yến
2019-07-05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2019-07-12,2.45,50.43,28.83,19.80,18.86,0.72,1.19,3.27,0.26,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2019-07-19,2.94,100.23,60.91,46.19,48.66,0.72,1.19,3.27,0.26,95.59,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2019-07-26,2.94,196.48,60.91,46.19,48.66,0.72,1.19,3.27,0.26,200.08,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2019-08-02,2.94,227.95,64.58,52.79,48.66,0.72,1.19,5.66,0.26,224.97,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-09,8.59,518.48,197.85,185.87,159.01,0.72,1.19,57.89,0.26,480.74,...,0.37,1.01,1.0,1.18,15.03,35.54,23.96,0.0,0.00,0.00
2024-08-16,8.59,521.79,197.85,185.87,159.01,0.72,1.19,57.89,0.26,480.74,...,0.37,1.01,1.0,1.18,15.03,35.54,23.96,2.7,0.00,0.00
2024-08-23,8.59,525.10,197.85,192.47,159.01,0.72,1.19,57.89,0.26,480.74,...,0.37,1.01,1.0,1.18,15.03,35.54,23.96,2.7,0.00,0.00
2024-08-30,8.59,528.41,197.85,199.07,159.01,0.72,1.19,57.89,0.26,480.74,...,0.37,1.01,1.0,1.18,15.03,35.54,23.96,2.7,0.05,0.00


In [23]:
# Drop last row
df_pivot_song = df_pivot_song[:-1]
df_pivot_song.shape, df_pivot_song.index[0], df_pivot_song.index[-1]

((270, 1493),
 Timestamp('2019-07-05 00:00:00'),
 Timestamp('2024-08-30 00:00:00'))

In [24]:
artists = df['artist'].unique().tolist()
print(len(artists))

542


In [25]:
duration_dict_artist = {song: [0 for _ in range(len(timestamps))] for song in artists}
duration_total_dict_artist = {song: 0 for song in artists}

In [26]:
time_idx = 1
for i, row in df.iterrows():
  while row['timestamp'] > timestamps[time_idx]:
    for s in artists:
      duration_dict_artist[s][time_idx] = duration_total_dict_artist[s]
    time_idx += 1

  artist = row['artist']
  duration_total_dict_artist[artist] += row['duration']
  duration_dict_artist[artist][time_idx] = duration_total_dict_artist[artist]

In [27]:
df_pivot_artist = pd.DataFrame(duration_dict_artist, index=timestamps)
print(len(df_pivot_artist), len(df_pivot_artist.columns))
df_pivot_artist

271 542


,Ariana Grande,Alan Walker,Ed Sheeran,ILLENIUM,Freddy Kalas,Post Malone,Billie Eilish,Taylor Swift,Rita Ora,benny blanco,...,Myra Trần,Lãng,Masew,Rapital,Arcade Player,TiHA Studio,HOYO-MiX,The Tokens,Ln,Dương Hoàng Yến
2019-07-05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2019-07-12,2.45,69.29,48.63,0.72,1.19,3.27,0.26,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2019-07-19,29.29,148.89,107.10,0.72,1.19,3.27,30.46,189.50,28.53,20.30,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2019-07-26,131.32,245.14,107.10,0.72,1.19,3.27,33.80,843.22,120.46,98.63,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2019-08-02,154.69,279.75,117.37,0.72,1.19,5.66,36.51,1044.62,138.41,113.13,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-09,897.12,910.76,1232.70,0.72,1.19,318.63,40.40,67094.99,392.27,389.34,...,4.51,0.18,2.5,0.01,15.03,35.54,23.96,0.0,0.00,0.00
2024-08-16,907.31,917.61,1245.28,0.72,1.19,318.63,40.40,67251.11,392.27,389.34,...,4.51,0.18,2.5,0.01,15.03,35.54,23.96,2.7,0.00,0.00
2024-08-23,917.20,920.92,1259.67,0.72,1.19,318.63,40.40,67399.49,392.27,389.36,...,4.51,0.18,2.5,0.01,15.03,35.54,23.96,2.7,0.00,0.00
2024-08-30,924.09,924.23,1266.27,0.72,1.19,318.63,40.40,67437.04,392.27,392.21,...,4.51,0.18,2.5,0.01,15.03,35.54,23.96,2.7,0.05,0.00


In [28]:
# Drop last row
df_pivot_artist = df_pivot_artist[:-1]
df_pivot_artist.shape, df_pivot_artist.index[0], df_pivot_artist.index[-1]

((270, 542),
 Timestamp('2019-07-05 00:00:00'),
 Timestamp('2024-08-30 00:00:00'))

In [29]:
# Check if values in each column are increasing
for col in df_pivot_artist.columns:
  prev_val = 0
  for i, val in enumerate(df_pivot_artist[col]):
    if val < prev_val:
      print(col)
      break
    prev_val = val

In [30]:
import bar_chart_race as bcr

# Create the bar chart race
bcr.bar_chart_race(
    df=df_pivot_song,
    title='Top 10 Most-played Songs (min)',
    filename='spotify_top_10_songs.mp4',  # Save as a file
    bar_label_size=17,
    tick_label_size=10,
    cmap='dark12',  # Color map for the bars
    figsize=(12, 6),
    period_length=400,  # Duration of each frame
    filter_column_colors=True,
    period_fmt='%b %-d, %Y',
    n_bars=10  # Limit to top 10 songs
)

In [31]:
# from IPython.display import Video

# # Display the video
# Video('scratch.mp4', embed=True)

In [32]:
# Create the bar chart race
bcr.bar_chart_race(
    df=df_pivot_artist,
    title='Top 10 Most-played Artists (min)',
    filename='spotify_top_10_artists.mp4',  # Save as a file
    bar_label_size=17,
    tick_label_size=10,
    cmap='dark12',  # Color map for the bars
    figsize=(12, 6),
    period_length=400,  # Duration of each frame
    filter_column_colors=True,
    period_fmt='%b %-d, %Y',
    n_bars=10  # Limit to top 10 songs
)

In [33]:
# pip install moviepy
# from moviepy.editor import VideoFileClip, concatenate_videoclips

# # Load the original video
# original_clip = VideoFileClip('final.mp4')

# # Extract the last frame
# last_frame = original_clip.get_frame(original_clip.duration)

# # Create a clip from the last frame
# from moviepy.editor import ImageClip
# last_frame_clip = ImageClip(last_frame).set_duration(5)  # Duration in seconds

# # Set the fps of the last frame clip to match the original video
# last_frame_clip = last_frame_clip.set_fps(original_clip.fps)

# # Concatenate the original video with the last frame clip
# extended_clip = concatenate_videoclips([original_clip, last_frame_clip])

# # Write the result to a new file
# extended_clip.write_videofile('top_10_extended.mp4')


Extract the data

In [49]:
df_pivot_artist.to_csv('df_pivot_artist.csv')
df_pivot_song.to_csv('df_pivot_song.csv')